In [1]:
#!/usr/bin/python
# Modified 1/15 by HYK, to generate cell lineage data
# dilution model:  plot after 
# simulating cell number distributions:  think of a way to plot this!
# Incorporate binomial partitioning, DONE!
# Cumulative addition of time over multiple rounds of simulation, DONE!
# First let system reach steady state
# Generalized functions for simulating arbitrary reaction network topologies
# Simulation cell number distributions as well, for multiple runs
# Also, write a MATLAB script to rapidly generate plots from the simulated data

"""
Created on Tue Dec 13 20:08:54 2016
@author: Sam Nguyen, HYK
"""
import roadrunner
import numpy 
from numpy import random as rand
from numpy import nonzero as nonzero
import math
from scipy import asarray as ar,exp
from multiprocessing import Pool
import csv
import random
import sys
roadrunner.Config.setValue(roadrunner.Config.MAX_OUTPUT_ROWS,10000)

# Tellurium loader
def loada(sbstr):
    import antimony as sb
    r = sb.loadAntimonyString(sbstr)
    if r < 0:
        raise RuntimeError('Failed to load Antimony model: {}'.format(sb.getLastError()))
    return roadrunner.RoadRunner(sb.getSBMLString(sb.getModuleNames()[-1]))

# Tellurium simulation function

# General simulation algorithm
# 1) Simulate the first cell, give it the first index
# 2) At the end, assign expression conditions to the two daugthers. Give the 2 daughter two new indices
# 3) Grow the indice list by adding newly indexed cells into the list
# 4) Save the index information to a cellinfo file. The simulation condition goes to the result file
# 5) Go to the next cell on the index list and start the simulation again

def doSimulation(tc,tmax,alpha_x1,alpha_x2,alpha_y):      
    r = loada("""
    # define cell cycling parameters
    TCycle = 10
    growth = 1
    initsize = 1
    V := (growth*(time/TCycle) + initsize)
    C = 0

    #define ode    
    
    R1: => x0 ; c0*V                   # this is the level of the chromatin regulator
    R2: x0 => ; d0*x0
  
    R3:  => px1 ; a1*x0*(1-px1)          # promoter activation of anterior hox gene
    R4: px1 => ; b1*x2*px1               # promoter inactivation (irreversible)
    R5:  => x1 ; c1*V*px1               # synthesis of X, rate increases with volume 
    R6: x1 => ;  d1*x1                  # degradation of X
    
    R7:  => px2 ; a2*x0*(1-px2)            # px2 promoter activation
    R8:   => x2 ; c2*V*px2
    R9: x2 =>   ; d2*x2

    R10:  => py ; ay2*(x2+x1)*(1-py)        # y is the cell cycle terminator
    R11:    => y ; cy*V*py
    R12: y =>   ;  dy*y
    
 
    # define initial values
    
    x0 = 1000
    px1 = 0
    x1 = 0
    px2 = 0
    x2 = 0
    py = 0
    y = 0
   
    # define rate constants:  modeling biological systems is cool!
    c0 = 10^3               # production rate of signal
    d0 = 1                  # degradation rate of signal
    a1 = 9*10^(-5)                # activation rate of anterior hox gene
    b1 = 0.1                # inactivation rate of anterior hox gene by posterior
    c1 = 10^3
    d1 = 1
    a2 = 1.2*10^(-4)               # this activation rate is influenced by the putative enhancer
    c2 = 850
    d2 = 1
    ay1 = 0              # 300 hrs would the termination step depend on the posterior hox gene segment?  Is there preservation of numbers?
    ay2 = 6*10^(-6)
    cy = 10^3
    dy = 1
    
    """)
    
    # set parameters of stochastic simulation
    r.integrator = 'gillespie'
    r.integrator.variable_step_size = False  
    r.TCycle = tc # define cell cycle time
    r.a1 = alpha_x1
    r.a2 = alpha_x2     # this is the activity of the enhancer
    r.ay2 = alpha_y
    
    ############################
    dt = 0.1;  # time interval for plotting
    result = []
    threshold = 500 # threshold concentration for y reaching activation threshold
    # what about this?  
    # we need to generate a cue
    
    #xinit = float(numpy.random.normal(603,24.37,1))   # parameters derived from steady state with no signal simulations 
    
    cellno = [[1]]         # keep track of all the cells
    terminal = [[0]]       # is this cell going to divide again?
    levels = [[r.x0, r.px1, r.x1, r.px2, r.x2, r.py, r.y]]     
    # initial Px, X and Y, Pz and Z values for levels of stuff
    simtime = [[0, tc]]   # time duration to simulate the next cell, start before t = 0
    children = []          # the children of this cell
    #ar(cellno),ar(children),ar(simtime),ar(terminal))
    cellcounter = 1       # this is the largest index for the cell
    counter = 0           # index for all the    
    
    while (counter < len(cellno)):     # there are still simulations in the cue 
        # update the initial conditions
        r.x0 = levels[counter][0]
        r.px1 = levels[counter][1]
        r.x1 = levels[counter][2]
        r.px2 = levels[counter][3]
        r.x2 = levels[counter][4]
        r.py = levels[counter][5]
        r.y = levels[counter][6]    
        r.C = int(cellno[counter][0])     # this is the cell number

        tstart = float(simtime[counter][0])   # start time 
        tend = float(simtime[counter][1])     # end time
        
        if (terminal[counter][0]==0):      # cell has not finished growing
            r.growth = 1
            r.initsize = 1
            r.k = 0.01            
            SubResult = r.simulate(0, (tend-tstart), int((tend-tstart)/dt),['C','time','V','x0','px1','x1','px2','x2','py','y'])

        elif (terminal[counter][0]==1):    # cell has finished growing, don't simulate 
            r.growth = 0
            r.initsize = 1
            r.k = 0
            SubResult = [[r.C, 0, 1, r.x0, r.px1, r.x1, r.px2, r.x2, r.py, r.y], [r.C, tend-tstart, 1, r.x0, r.px1, r.x1, r.px2, r.x2, r.py, r.y]]        
            # append the simulation results to the spreadsheet
        for row in SubResult:
            row[1] = row[1]+tstart
            result.append(row)
                
        # if simulation time exceeded max time
        if (simtime[counter][1] >= tmax):      # maximum simulation time has been reached
            children = numpy.append(children,[[-1,-1]],axis=0)  # indicate no more children
        else:
            sr2 = ar(SubResult)      # sub result in array format
            
            # calculate whether threshold has been reached for 
            yc = sr2[:,9]/sr2[:,2]     # the concentration of y, the cell cycle terminator
            yt = (yc>threshold)      # we need to change the distribution of the
            tyt = nonzero(yt)        # index of the transition the transition time

            x1c = sr2[:,5]/sr2[:,2]     # the concentration of x1, the cell cycle terminator
            x1t = (x1c>threshold)      # we need to change the distribution of the
            tx1t = nonzero(x1t)        # index of the transition the transition time

            x2c = sr2[:,7]/sr2[:,2]     # the concentration of x2, the cell cycle terminator
            x2t = (x2c>threshold)      # we need to change the distribution of the
            tx2t = nonzero(x2t)        # index of the transition the transition time
            
            if (len(tyt[0])>0):         # a transition occurred, kill the cell
                if (len(children)==0):
                    children = [[-1,-1]]
                else:
                    children = numpy.append(children,[[-1,-1]],axis=0)
            else: # transition did not occur, do asymmetric cell division with second cell stopping
                if (len(children)==0):   # generate two children
                    children = [[cellcounter+1,cellcounter+2]]
                else:   
                    children = numpy.append(children, [[cellcounter+1,cellcounter+2]],axis=0)
                x0d = float(rand.binomial(r.x0, 0.5))    # random partitioning of proteins
                x1d = float(rand.binomial(r.x1, 0.5))
                x2d = float(rand.binomial(r.x2, 0.5))
                yd = float(rand.binomial(r.y, 0.5))
                cellno = numpy.append(cellno, [[cellcounter+1],[cellcounter+2]],axis=0)     
                levels = numpy.append(levels, [[x0d, r.px1, x1d, r.px2, x2d, r.py, yd], [r.x0-x0d, r.px1, r.x1-x1d, r.px2, r.x2-x2d, r.py, r.y-yd]], axis=0) 
                
                if (len(tx1t[0]>0)) or (len(tx2t[0]>0)):    # asymmetric cell division
                    if (tend+tc <= tmax):    # determine the end time for these dividing tracks
                        simtime = numpy.append(simtime,[[tend,tend+tc],[tend,tmax]],axis=0)
                    else:
                        simtime = numpy.append(simtime,[[tend,tmax],[tend,tmax]],axis=0)
                    terminal = numpy.append(terminal,[[0],[1]],axis=0)                         
                else:     # symmetric stem cell expansion
                    if (tend+tc <= tmax):    # determine the end time for these dividing tracks
                        simtime = numpy.append(simtime,[[tend,tend+tc],[tend,tend+tc]],axis=0)
                    else:
                        simtime = numpy.append(simtime,[[tend,tmax],[tend,tmax]],axis=0)
                    terminal = numpy.append(terminal,[[0],[0]],axis=0)         
                cellcounter = cellcounter+2   # append to the current cell counter
        counter = counter+1    # update the counter, go to next iteration of the while loop
    result = ar(result)    
    cellinfo = numpy.hstack((ar(cellno),ar(children),ar(simtime),ar(terminal)))
    cellinfo = numpy.vstack((cellinfo, ar([[-1, -1, -1, -1, -1, -1]])))
    return [result,cellinfo]
  
#Write csv file
def writeCSV(filename,result):
    with open(filename,'w') as mycvsfile:
        thedatawriter=csv.writer(mycvsfile)   
        thedatawriter.writerows(result)


In [ ]:

ay_range_log = numpy.array([-7,-6,-5,-4, -3])
ax1_range_log = numpy.array([-7.2, -6. , -5.2, -4.4, -3.6,-2.8])+2
ax2_range_log = numpy.array([-7.2, -6. , -5.2, -4.4, -3.6,-2.8])+2

ay_range = 10.0**ay_range_log  # rate of production of chromatin regulator
ax1_range = 10.0**ax1_range_log
ax2_range = 10.0**ax2_range_log;
for ay in ay_range:
    for ax1 in ax1_range:
        for ax2 in ax2_range:
            
            print('ay {}'.format(ay))
            print('ax1 {}'.format(ax1))
            print('ax2 {}'.format(ax2))
            
            ay_log = numpy.log10(ay)
            ax1_log = numpy.log10(ax1)
            ax2_log = numpy.log10(ax2)
            
            for i in range(0,400,1):  # do multiple repeats 
                
                output = doSimulation(10,1000,ax1,ax2,ay)    # simulate for longer duration to encompass switching time
                result = output[0]
                cellinfo = output[1]   # information about the cell
                
                repcount = i
                print(repcount)
                writeCSV('/external/Sam/JRSI/HoxGeneNetWork/data/data_ax1_{}_ax2_{}_y_{}_repeat_{}.csv'.format(round(ax1_log,2),round(ax2_log,2),round(ay_log,2),repcount), result)
                writeCSV('/external/Sam/JRSI/HoxGeneNetWork/data/cellinfo_ax1_{}_ax2_{}_y_{}_repeat_{}.csv'.format(round(ax1_log,2),round(ax2_log,2),round(ay_log,2),repcount), cellinfo)
